## Задача:

Построить модель машинного обучения, предсказывающую траты у посетителей магазина.

## План решения:

1. Загрузите данные и подготовьте их:

- сделайте из текстового признака, обозначающего пол, бинарный флаг 0/1

- переименуйте признаки так, чтобы в названиях не было пробелов.

In [56]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor, export_text

In [57]:
df = pd.read_csv('Mall_Customers.csv')
df.head()

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [58]:
df['Genre'] = df['Genre'].map({'Male': 1, 'Female': 0})
df.Genre.value_counts()

Genre
0    112
1     88
Name: count, dtype: int64

In [59]:
df.columns = ['CustomerID', 'Genre', 'Age', 'AnnualIncome', 'SpendingScore']
df.columns

Index(['CustomerID', 'Genre', 'Age', 'AnnualIncome', 'SpendingScore'], dtype='object')

2. Доработайте функцию построения сплита (разработанную в рамках скринкаста):

a. Занесите расчет хаотичности целевого признака в выборке в саму функцию построения сплита

b. Уберите промежуточные выводы

c. В качестве результата отработки функции возвращайте пару “признак”+”значение для сплита”, при которых достигается наилучший прирост информации. Если таких не нашлось (если выборка однородна с точки зрения всех признаков и ее невозможно разделить), возвращайте, например, None.

d. Сделаем функцию похожей на библиотечную: пусть она принимает на вход отдельно датафрейм с признаками (назовите параметр X ) и столбец со значениями целевого признака (назовите y). Доработайте код функции под новые входные данные. Подсказка: чтобы понять, какие значения целевого признака будут принимать объекты, попавшие в левую/правую часть подвыборки, создайте маску из столбца с фичей и примените ее к столбца с целевым признаком:

- Чтобы получить маску, к столбцу в признаком, по которому происходит деление примените операцию сравнения со значением, по которому хотите выполнить сплит. Получится столбец из True/False длины, равной количеству элементов в исходной выборке, подлежащей сплиту.

- Примените полученную маску к столбцу с целевыми значениями target[mask]. В итоге останутся только значения целевой переменной для наблюдений, которым в маске соответствовало значение True.

In [60]:
features = df[['Genre', 'Age', 'AnnualIncome']]
target = df['SpendingScore']

In [61]:
def calc_best_split(X, y):
    max_information_gain = 0
    best_split_feature = None
    best_split_value = None

    # рассчитываем исходный разброс
    init_criterion_value = np.var(y)
    
    # перебираем все признаки
    for feature in X.columns:
        
        # составляем значения для сплитов по признаку
        diff_feature_values = sorted(set(X[feature])) #[0, 1]
        
        # перебираем возможные сплиты по признаку
        for i in range(len(diff_feature_values) - 1):
            
            information_gain = init_criterion_value
            
            # считаем значение сплита
            split_value = (diff_feature_values[i] + diff_feature_values[i+1]) / 2
            
            # считаем хаотичность в левой группе
            values_in_group_mask = X[feature] <= split_value
            values_in_group = y[values_in_group_mask]
            criterion_value_for_group = np.var(values_in_group)
            ratio_for_group = len(values_in_group) / len(y)
            information_gain -= criterion_value_for_group * ratio_for_group
            
            # считаем хаотичность в правой группе
            values_in_group = y[~values_in_group_mask]
            criterion_value_for_group = np.var(values_in_group)
            ratio_for_group = len(values_in_group) / len(y)
            information_gain -= criterion_value_for_group * ratio_for_group
            
            #считаем прирост иноформации
            if max_information_gain < information_gain:
                max_information_gain = information_gain
                best_split_feature = feature
                best_split_value = split_value
                
    return best_split_feature, best_split_value

3. Реализуйте функцию построения решающего дерева fit_regression_tree, допускающую ограничения на максимальную глубину дерева, а также минимальное число наблюдений в выборке, подлежащей сплиту. В результате запуска функции должны быть выведены правила, по которым должен происходить сплит, а в листьях - предикт (то есть среднее значение целевого признака на подвыборке обучающего датафйрема, попавшей в этот лист). Бинарное дерево строится рекурсивной, поэтому функция его простроения тоже будет рекурсивной:

a. Реализуйте часть для выхода из рекурсии. Здесь должна выполняться проверка на достижение максимально допустимой глубины дерева/достаточное количество наблюдений в выборке для выполнения очередного сплита/возможность построения сплита. Невозможно построить сплит, если все объекты в выборке одинаковы с точки зрения значений фичей. Если одно из этих условий выполняется, выводите предикт в следующем формате:
"| " * <текущая глубина дерева>"|---" value: <предикт с 2 знаками после запятой>]'

b. Реализуйте альтернативный путь для дальнейшего построения дерева:

- Выведите правило, по которому наблюдения “идут влево” в формате:
"| " * <текущая глубина дерева>"|---" <фича для сплита> <= <значение для сплита>

- Постройте сплит “пошедшей влево” подвыборки

- Выведите правило, по которому наблюдения “идут вправо” в формате:
"| " * <текущая глубина дерева>"|---" <фича для сплита> > <значение для сплита>

- Постройте сплит “пошедшей вправо” подвыборки.

In [62]:
def fit_regression_tree(X, y, depth=0, max_depth=None, min_samples_split=2):

    # прписываем ограничения:
    if (max_depth is not None and depth == max_depth) |\
            (len(X) < min_samples_split) |\
            (len(set(y)) == 1): 
        print("|   " * depth + "|--- value: [{:.2f}]".format(y.mean()))
        return

    feature, threshold = calc_best_split(X, y)
    if feature is None:
        print("|   " * depth + "|--- value: [{:.2f}]".format(y.mean()))
        return

    left_mask = X[feature] <= threshold

    left_X = X[left_mask]
    left_y = y[left_mask]
    right_X = X[~left_mask]
    right_y = y[~left_mask]
    
    print("|   " * depth + "|--- {} <= {:.2f}".format(feature, threshold))
    fit_regression_tree(left_X, left_y, depth+1, max_depth, min_samples_split)
    
    print("|   " * depth + "|--- {} > {:.2f}".format(feature, threshold))
    fit_regression_tree(right_X, right_y, depth+1, max_depth, min_samples_split)

4. Проверьте корректность реализованного алгоритма

a. Убедитесь, что без ошибок строится дерево без ограничений (то есть до последнего возможного сплита)

b. Убедитесь, что без ошибок отрабатывает алгоритм для построения дерева с ограничением на глубину. Выберите максимально допустимую глубину, равную 2. Также постройте дерево с помощью библиотечного алгоритма (DecisionTreeRegressor) на тех же входных данных. Выведите его с помощью функции export_text и сравните построенные деревья. Они должны совпадать.

c. Убедитесь, что без ошибок отрабатывает алгоритм для построения дерева с ограничением на минимальное количество элементов в выборке, над которой производится в сплит. Также сравните построенное дерево с деревом, построенным с помощью библиотечной реализации, при ограничении min_samples_split=70.

In [63]:
# построение дерева без ограничений
fit_regression_tree(features, target)

|--- Age <= 39.50
|   |--- Age <= 20.50
|   |   |--- AnnualIncome <= 69.00
|   |   |   |--- AnnualIncome <= 18.50
|   |   |   |   |--- Genre <= 0.50
|   |   |   |   |   |--- value: [6.00]
|   |   |   |   |--- Genre > 0.50
|   |   |   |   |   |--- value: [39.00]
|   |   |   |--- AnnualIncome > 18.50
|   |   |   |   |--- AnnualIncome <= 41.50
|   |   |   |   |   |--- Age <= 19.00
|   |   |   |   |   |   |--- value: [92.00]
|   |   |   |   |   |--- Age > 19.00
|   |   |   |   |   |   |--- Genre <= 0.50
|   |   |   |   |   |   |   |--- value: [75.00]
|   |   |   |   |   |   |--- Genre > 0.50
|   |   |   |   |   |   |   |--- value: [66.00]
|   |   |   |   |--- AnnualIncome > 41.50
|   |   |   |   |   |--- AnnualIncome <= 53.50
|   |   |   |   |   |   |--- AnnualIncome <= 47.00
|   |   |   |   |   |   |   |--- value: [55.00]
|   |   |   |   |   |   |--- AnnualIncome > 47.00
|   |   |   |   |   |   |   |--- value: [59.00]
|   |   |   |   |   |--- AnnualIncome > 53.50
|   |   |   |   |   |   |

In [64]:
# построение дерева с ограничением на максимальную глубину max_depth=2
fit_regression_tree(features, target, max_depth=2)

|--- Age <= 39.50
|   |--- Age <= 20.50
|   |   |--- value: [44.65]
|   |--- Age > 20.50
|   |   |--- value: [62.53]
|--- Age > 39.50
|   |--- AnnualIncome <= 72.00
|   |   |--- value: [41.77]
|   |--- AnnualIncome > 72.00
|   |   |--- value: [19.79]


In [65]:
# построение дерева с ограничением min_samples_split=70
fit_regression_tree(features, target, min_samples_split=70)

|--- Age <= 39.50
|   |--- Age <= 20.50
|   |   |--- value: [44.65]
|   |--- Age > 20.50
|   |   |--- Age <= 32.50
|   |   |   |--- value: [66.59]
|   |   |--- Age > 32.50
|   |   |   |--- value: [55.09]
|--- Age > 39.50
|   |--- AnnualIncome <= 72.00
|   |   |--- value: [41.77]
|   |--- AnnualIncome > 72.00
|   |   |--- value: [19.79]


In [66]:
# построение библиотечного дерева без ограничений
model = DecisionTreeRegressor()
model.fit(features, target)

tree_rules = export_text(model, feature_names=list(features.columns))
print(tree_rules)

|--- Age <= 39.50
|   |--- Age <= 20.50
|   |   |--- AnnualIncome <= 69.00
|   |   |   |--- AnnualIncome <= 18.50
|   |   |   |   |--- Age <= 19.50
|   |   |   |   |   |--- value: [39.00]
|   |   |   |   |--- Age >  19.50
|   |   |   |   |   |--- value: [6.00]
|   |   |   |--- AnnualIncome >  18.50
|   |   |   |   |--- AnnualIncome <= 41.50
|   |   |   |   |   |--- Age <= 19.00
|   |   |   |   |   |   |--- value: [92.00]
|   |   |   |   |   |--- Age >  19.00
|   |   |   |   |   |   |--- Genre <= 0.50
|   |   |   |   |   |   |   |--- value: [75.00]
|   |   |   |   |   |   |--- Genre >  0.50
|   |   |   |   |   |   |   |--- value: [66.00]
|   |   |   |   |--- AnnualIncome >  41.50
|   |   |   |   |   |--- AnnualIncome <= 53.50
|   |   |   |   |   |   |--- AnnualIncome <= 47.00
|   |   |   |   |   |   |   |--- value: [55.00]
|   |   |   |   |   |   |--- AnnualIncome >  47.00
|   |   |   |   |   |   |   |--- value: [59.00]
|   |   |   |   |   |--- AnnualIncome >  53.50
|   |   |   |   |   

In [67]:
# построение библиотечного дерева с ограничением на максимальную глубину max_depth=2
model = DecisionTreeRegressor(max_depth=2)
model.fit(features, target)

tree_rules = export_text(model, feature_names=list(features.columns))
print(tree_rules)

|--- Age <= 39.50
|   |--- Age <= 20.50
|   |   |--- value: [44.65]
|   |--- Age >  20.50
|   |   |--- value: [62.53]
|--- Age >  39.50
|   |--- AnnualIncome <= 72.00
|   |   |--- value: [41.77]
|   |--- AnnualIncome >  72.00
|   |   |--- value: [19.79]



In [68]:
# построение библиотечного дерева с ограничением min_samples_split=70
model = DecisionTreeRegressor(min_samples_split=70)
model.fit(features, target)

tree_rules = export_text(model, feature_names=list(features.columns))
print(tree_rules)

|--- Age <= 39.50
|   |--- Age <= 20.50
|   |   |--- value: [44.65]
|   |--- Age >  20.50
|   |   |--- Age <= 32.50
|   |   |   |--- value: [66.59]
|   |   |--- Age >  32.50
|   |   |   |--- value: [55.09]
|--- Age >  39.50
|   |--- AnnualIncome <= 72.00
|   |   |--- value: [41.77]
|   |--- AnnualIncome >  72.00
|   |   |--- value: [19.79]



### Вывод:

Алгоритм функции fit_regression_tree построения решающего дерева работает без ошибок, как при наличии ограничений, так и без них. Все полученные результаты полностью совпадают с результатами, полученными с помощью библиотеки DecisionTreeRegressor.